In [1]:
%gui qt

In [2]:
from typing import Optional, Tuple, Union

import starfish
from starfish.types import Axes
from starfish import FieldOfView, IntensityTable
import xarray as xr
import numpy as np
import trackpy

In [3]:
def percent_clip_to_zero(image: Union[xr.DataArray, np.ndarray],
                 p_min: int, p_max: int) -> np.ndarray:
        """Clip values under minimum percentile of an image to zero"""
        v_min, v_max = np.percentile(image, [p_min, p_max])
        image = image.clip(min=v_min, max=v_max) - np.float32(v_min)
        return image

def clip_to_zero(image: Union[xr.DataArray, np.ndarray],
                 minval=None, maxval=None) -> np.ndarray:
    image = image.clip(min=minval, max=maxval) - np.float32(minval)
    return image

# gaussian blur to smooth z-axis
z_gauss_filt = starfish.image.Filter.GaussianLowPass(sigma=(1, 0, 0), is_volume=True)

In [4]:
def preprocess_survey_tiles(experiment: starfish.Experiment,
                            fov_name: str,
                            n_processes: Optional[int]=None):
    fov = experiment[fov_name]
    images = enumerate(fov.iterate_image_type(FieldOfView.PRIMARY_IMAGES))
    for image_number, primary_image in images:
        primary_image.apply(func=percent_clip_to_zero, group_by={Axes.CH, Axes.ZPLANE}, p_min=75, p_max=100, in_place=True)
        primary_image.apply(func=trackpy.bandpass, group_by={Axes.CH, Axes.ZPLANE}, lshort=0.5, llong=7, threshold=1/65535, in_place=True)
        primary_image.apply(func=clip_to_zero, group_by={Axes.CH, Axes.ZPLANE}, minval=1/65535, in_place=True)
        z_gauss_filt.run(primary_image, in_place=True)
        return primary_image

In [5]:
experiment = starfish.Experiment.from_json("/Users/brian/work/data/the_other_other/czi_mouse_single_round_SpaceTx/experiment.json")
preprocessed = preprocess_survey_tiles(experiment, 'fov_000')

100%|██████████| 102/102 [00:11<00:00,  8.78it/s]


In [8]:
preprocessed.to_multipage_tiff('/Users/brian/work/data/the_other_other/preprocessed_fov_000')

In [7]:
preprocessed.to_multipage_tiff?

In [8]:
%matplotlib qt
import matplotlib.pyplot as plt
import skimage.io

starfish_volume = skimage.io.imread("/home/njmei/Documents/czi_mouse_single_round_SpaceTx_2/preprocessed_fov_000.TIFF")
print(starfish_volume.shape)

survey_ch0 = skimage.io.imread("/home/njmei/Documents/czi_mouse_single_round/1-Pos_003_006/processedData/1-Pos_003_006_Bandpass488_036processedStack.tif")
survey_ch1 = skimage.io.imread("/home/njmei/Documents/czi_mouse_single_round/1-Pos_003_006/processedData/1-Pos_003_006_Bandpass561_036processedStack.tif")
survey_ch2 = skimage.io.imread("/home/njmei/Documents/czi_mouse_single_round/1-Pos_003_006/processedData/1-Pos_003_006_Bandpass640_036processedStack.tif")
print(survey_ch0.shape)

# Determine differences between survey processed data vs starfish processed data
ui16 = np.iinfo(np.uint16)

scaled_survey_tile = np.divide(survey_ch0[0], ui16.max)
starfish_tile = starfish_volume[0,:,:,0]

diff = scaled_survey_tile - starfish_tile

# Plot intensity comparisons
fig, ax = plt.subplots(nrows=2, ncols=2)
ax = ax.ravel()
ax[0].imshow(scaled_survey_tile)
ax[0].set_title("scaled_survey_tile")
ax[1].imshow(starfish_tile)
ax[1].set_title("starfish_tile")
ax[2].imshow(diff)
ax[2].set_title("difference\n(survey - starfish)")
ax[3].imshow(np.abs(diff))
ax[3].set_title("absolute diff\nabs(survey - starfish)")
fig.tight_layout()
print(f"diff max: {diff.max()}")
print(f"diff min: {diff.min()}")
print(f"abs max: {np.abs(diff).max()}")

# Plot intensity histogram comparisons
fig, ax = plt.subplots(nrows=2, ncols=1, sharey=True, sharex=True)
_ = ax[0].hist(scaled_survey_tile.ravel(), bins=200)
_ = ax[0].set_title("histogram of scaled survey intensities")
_ = ax[1].hist(starfish_tile.ravel(), bins=200)
_ = ax[1].set_title("histogram of starfish intensities")
fig.tight_layout()

(37, 2048, 2048, 3)
(37, 2048, 2048)
diff max: 0.00033250445282462476
diff min: -1.8377792002866045e-05
abs max: 0.00033250445282462476


In [ ]:
import matplotlib.pyplot as plt
import skimage.io

# Check that unprocessed tiffs between starfish and our pipeline are the same
starfish_tile = skimage.io.imread("/home/njmei/Documents/czi_mouse_single_round_SpaceTx_2/primary_images-fov_000-Z0-H0-C0.tiff")
original_tile = skimage.io.imread("/home/njmei/Documents/czi_mouse_single_round/1-Pos_003_006/1-Pos_003_006_Bandpass488_000.tif")
diff = original_tile - starfish_tile
print(f"original_tile - starfish_tile max difference: {diff.max()}")
print(f"starfish tile max value: {original_tile.max()}, original tile min value: {original_tile.min()}")
print(f"original tile max value: {original_tile.max()}, original tile min value: {original_tile.min()}")